In [1]:
import numpy as np
from sklearn.base import BaseEstimator,TransformerMixin, ClassifierMixin
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import ElasticNetCV, LassoLarsCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline, make_union
from sklearn.utils import check_array
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import r2_score


In [2]:
import os

import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from math import pi
import seaborn as sns
%matplotlib inline 

import warnings
warnings.filterwarnings("ignore")

# Print all rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
IS_LOCAL = True

import os

if IS_LOCAL:
    PATH="input"
else:
    PATH="../input"
print(os.listdir(PATH))

['train.csv.zip', 'test.csv', 'sample_submission.csv.zip', 'test.csv.zip', 'train.csv', 'sample_submission.csv']


In [3]:
#train = pd.read_csv('../input/train.csv')
#test = pd.read_csv('../input/test.csv')

train_df = pd.read_csv(os.path.join(PATH, "train.csv"))
test_df = pd.read_csv(os.path.join(PATH, "test.csv"))

In [5]:
train_df.drop(['ID'], axis=1, inplace=True)
test_df.drop(['ID'], axis=1, inplace=True)

train_df['target'] = np.log1p(train_df.target)


In [7]:
train = train_df
test = test_df

In [8]:
# https://www.kaggle.com/hakeem/stacked-then-averaged-models-0-5697

import numpy as np
from sklearn.base import BaseEstimator,TransformerMixin, ClassifierMixin
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import ElasticNetCV, LassoLarsCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline, make_union
from sklearn.utils import check_array
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import r2_score



class StackingEstimator(BaseEstimator, TransformerMixin):
    
    def __init__(self, estimator):
        self.estimator = estimator

    def fit(self, X, y=None, **fit_params):
        self.estimator.fit(X, y, **fit_params)
        return self
    def transform(self, X):
        X = check_array(X)
        X_transformed = np.copy(X)
        # add class probabilities as a synthetic feature
        if issubclass(self.estimator.__class__, ClassifierMixin) and hasattr(self.estimator, 'predict_proba'):
            X_transformed = np.hstack((self.estimator.predict_proba(X), X))

        # add class prodiction as a synthetic feature
        X_transformed = np.hstack((np.reshape(self.estimator.predict(X), (-1, 1)), X_transformed))

        return X_transformed


# local R2
# n_comp = 6 ;  R2 score on train data: 0.65089225245 ; LB: 0.55577
# n_comp = 12;  R2 score on train data: 0.65950822961 ; LB: 0.56760
# n_comp = 16;  R2 score on train data: 0.65799524004 ; LB: 0.56317
# n_comp = 20;  R2 score on train data: 0.66681870314 ; LB: 0.56262
# n_comp = 40;  R2 score on train data: 0.67135596029 ; LB: 0.55842
# n_comp = 80;  R2 score on train data: 0.67589753862
# n_comp = 160; R2 score on train data: 0.68492424399 : LB: 0.55897
# n_comp = 240; R2 score on train data: 0.69159326043 ; LB: 
# n_comp = 320; R2 score on train data: 0.69908510068 ; LB: 

n_comp = 320

# tSVD
print('tsvd')
tsvd = TruncatedSVD(n_components=n_comp, random_state=420)
tsvd_results_train = tsvd.fit_transform(train.drop(["target"], axis=1))
tsvd_results_test = tsvd.transform(test)

# PCA
print('pca')
pca = PCA(n_components=n_comp, random_state=420)
pca2_results_train = pca.fit_transform(train.drop(["target"], axis=1))
pca2_results_test = pca.transform(test)

# ICA
print('ica')
ica = FastICA(n_components=n_comp, random_state=420)
ica2_results_train = ica.fit_transform(train.drop(["target"], axis=1))
ica2_results_test = ica.transform(test)

# GRP
print('grp')
grp = GaussianRandomProjection(n_components=n_comp, eps=0.1, random_state=420)
grp_results_train = grp.fit_transform(train.drop(["target"], axis=1))
grp_results_test = grp.transform(test)

# SRP
print('srp')
srp = SparseRandomProjection(n_components=n_comp, dense_output=True, random_state=420)
srp_results_train = srp.fit_transform(train.drop(["target"], axis=1))
srp_results_test = srp.transform(test)

#save columns list before adding the decomposition components

usable_columns = list(set(train.columns) - set(['target']))

# Append decomposition components to datasets
for i in range(1, n_comp + 1):
    train['pca_' + str(i)] = pca2_results_train[:, i - 1]
    test['pca_' + str(i)] = pca2_results_test[:, i - 1]

    train['ica_' + str(i)] = ica2_results_train[:, i - 1]
    test['ica_' + str(i)] = ica2_results_test[:, i - 1]

    train['tsvd_' + str(i)] = tsvd_results_train[:, i - 1]
    test['tsvd_' + str(i)] = tsvd_results_test[:, i - 1]

    train['grp_' + str(i)] = grp_results_train[:, i - 1]
    test['grp_' + str(i)] = grp_results_test[:, i - 1]

    train['srp_' + str(i)] = srp_results_train[:, i - 1]
    test['srp_' + str(i)] = srp_results_test[:, i - 1]

#usable_columns = list(set(train.columns) - set(['y']))

y_train = train['target'].values
y_mean = np.mean(y_train)
#id_test = test['ID'].values

#finaltrainset and finaltestset are data to be used only the stacked model (does not contain PCA, SVD... arrays) 
finaltrainset = train[usable_columns].values
finaltestset = test[usable_columns].values



print('xgb matrices')
dtrain = xgb.DMatrix(train.drop('target', axis=1), y_train)
dtest = xgb.DMatrix(test)

tsvd
pca
ica
grp
srp


In [10]:
'''Train the xgb model then predict the test data'''

xgb_params = {
    'n_trees': 520, 
    'eta': 0.0045,
    'max_depth': 4,
    'subsample': 0.93,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    #'silent': 1
}
# NOTE: Make sure that the class is labeled 'class' in the data file


num_boost_rounds = 1250
# train model
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)
y_pred = model.predict(dtest)

'''Train the stacked models then predict the test data'''

stacked_pipeline = make_pipeline(
    StackingEstimator(estimator=LassoLarsCV(normalize=True, verbose=True)),
    StackingEstimator(estimator=GradientBoostingRegressor(
        learning_rate=0.001, loss="huber", max_depth=3, 
        max_features=0.55, min_samples_leaf=18, min_samples_split=14, 
        subsample=0.7, verbose=1)),
    LassoLarsCV()

)

print('train')
stacked_pipeline.fit(finaltrainset, y_train)
results = stacked_pipeline.predict(finaltestset)


train


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  3.5min finished


      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.5330          -0.0056           27.94s
         2           1.5437           0.0005           27.60s
         3           1.4938           0.0005           26.79s
         4           1.5183           0.0004           26.59s
         5           1.5043           0.0005           26.32s
         6           1.4929           0.0005           25.83s
         7           1.4948           0.0006           25.64s
         8           1.5322           0.0003           25.23s
         9           1.5004           0.0003           25.00s
        10           1.4808           0.0004           24.63s
        20           1.5036           0.0004           21.80s
        30           1.4811           0.0005           19.02s
        40           1.4507           0.0004           16.15s
        50           1.4893           0.0003           13.47s
        60           1.4797           0.0005           10.83s
       

In [11]:
'''Score on the entire Train data when averaging'''
from sklearn.metrics import mean_squared_error

rmsle_stacked = mean_squared_error(y_train, stacked_pipeline.predict(finaltrainset)*0.2855 + model.predict(dtrain)*0.7145)
print('rmsle score on train data:', rmsle_stacked)



rmsle score on train data: 1.4192415074928373


In [ ]:
'''Average the preditionon test data  of both models then save it on a csv file'''

sub = pd.DataFrame()
sub['ID'] = id_test
sub['target'] = y_pred*0.75 + results*0.25
sub.to_csv('stacked-models.csv', index=False)
print('Finished.')

# Any results you write to the current directory are saved as output